In [2]:
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline
import os
import scipy

# Functions

An npz data file contains: ['time1', 'time2', 'timeoffset', 'timescale', 'voltscale', 'voltoffset', 'raw_data_chan1', 'raw_data_chan2']

In [3]:
def mean_maker(name, chan1 = False, chan2 = False): # only one channel at a time is callable atm because i think otherwise i have to also put that whole shebag in the frame and i dont want that
    fn = folder + name
    data = np.load(fn)
#     lst = data.files
    if chan1 :
        return np.mean(data['raw_data_chan1'])
    elif chan2:
        return np.mean(data['raw_data_chan2'])

In [4]:
def std_dev(name, chan1 = False, chan2 = False): # only one channel at a time is callable atm because i think otherwise i have to also put that whole shebag in the frame and i dont want that
    fn = folder + name
    data = np.load(fn)
#     lst = data.files
    if chan1 :
        return np.std(data['raw_data_chan1'])
    elif chan2:
        return np.std(data['raw_data_chan2'])

In [5]:
def find_change_points(data, threshold):
    # Calculate the gradient of the data
    gradient = np.gradient(data)

    # Find positions where the absolute gradient exceeds the threshold
    change_points = np.where(np.abs(gradient) > threshold)[0]

    return change_points

In [6]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), "valid") / w

In [7]:
def find_range(EF_data, photo_data, threshold=20):
#     EF_data = moving_average(EF_data, 2)
    points = find_change_points(EF_data, threshold)
    cut_ratio = points/EF_data.size
    photo_cut = photo_data[int(photo_data.size*min(cut_ratio)):int(photo_data.size*max(cut_ratio))]
    background_cut = photo_data[0:int(photo_data.size*min(cut_ratio))]
    return photo_cut, background_cut

In [8]:
def get_data(name, chan):
    fn = folder + name
    data = np.load(fn)
    return data['raw_data_chan'+str(chan)]

In [9]:
def get_fulldata(name):
    fn = folder + name
    data = np.load(fn)
    return data

In [10]:
def new_array(data):
    #for some reason the array loaded from the npz SUCKS, maybe someone can help and do it better
    #here I just create a new numpy array that can be used better!!!
    data_new = np.zeros(data.size)
    for i in range(data.size):
        data_new[i] = data[i]
    return data_new

In [11]:
def voltage_converter(data, voltoffset, voltscale):
#     # first, we convert the data such that the middle (125 pts) is at 0V
#     data = data - 125
#     #then we subtract the voltage offset converted into points, one unit of voltscale = -25 points
#     data = data - ((voltoffset/voltscale) * -25)
#     #then we convert from points to voltage
#     data = data / -25
#     #then we adjust the scale
#     data = data * voltscale

    data_new = (((data - 125) - ((voltoffset/voltscale) * -25))/ -25) * voltscale
    return data_new

In [12]:
def convert_all(name):
    data = get_fulldata(name)
    chan1 = data['raw_data_chan1']
    chan2 = data['raw_data_chan2']
    #Data saved before 04/11 has inverted volt offset and volt scale
    vs = data['voltscale']
    voff = data['voltoffset']
    time1 = data['time1']
    time2 = data['time2']
    toff = data['timeoffset']
    tscale = data['timescale']

    chan1 = new_array(chan1)
    chan2 = new_array(chan2)
    chan1 = voltage_converter(chan1, voff[0], vs[0])
    chan2 = voltage_converter(chan2, voff[1], vs[1])

    return chan1, chan2, time1, time2

In [13]:
#the input of data needs to be in voltage and spits out the power measured by the RF channel of the photodiode
def power_converter(data):
    R = 0.3 #A/W
    G = 50 *10^3 #V/A
    new_data = data/(R*G) #Returns power in Watt
    return new_data


In [14]:
# We start with 510 Watt, so the question is, how much do we lose? I don't think this is very valueable as a way to look at it , but it is possible
def find_effect(data):
    #how much do we lose through the first filter?
    theta = 170-90
    od = 0.0148 * theta
    i = 510 * (10**(-od))
    #Now this gets split in two
    i_ref = i_s = i/2
    #our sample beam also looses through the dichoric mirrors
    i_s = i_s *0.92*0.92
    #what we actually measure is the difference between our reference and our sample
    i_tot = i_ref - i_s
    #Now lets see how much higher the difference is in our data!
    new_data = data - i_tot
    return new_data


# fit data

In [52]:
def fit_cos_squared(x_data, y_data):
    # Initial guess for the parameters A, B, C
    params_covariance = 50
    for i in range(1,3):
        initial_guess = [1, i, 0]
        # Use curve_fit to find the best fit parameters
        params_1, params_covariance_1 = scipy.optimize.curve_fit(cos_squared, x_data, y_data, p0=initial_guess)
        print(np.sum(params_covariance_1))
        print(i)
        if (np.sum(params_covariance_1) < params_covariance):
            params = params_1
            params_covariance = np.sum(params_covariance_1)
    return params

In [16]:
def cos_squared(x, A, B, C):
    """Cosine squared function."""
    return A * np.cos(B * np.radians((x + C)))**2

In [17]:
def intensity_norm(data):
    min = np.min(data)
    if min < 0 :
        data = data + np.abs(min)
    max = np.max(data)
    return data/max

In [18]:
def plot_fit(x_data, y_data, params):
    """Plots the original data and the fitted curve."""
    plt.scatter(x_data, y_data, label='Data')
    plt.plot(x_data, cos_squared(x_data, *params), label='Fitted function', color='red')
#     formula_text = f'$y = {params[0]:.2f}  \cdot \cos^2( {params[1]:.2f} \cdot x + {params[2]:.2f})$'
#     plt.text(0.05, 0.95, formula_text, transform=plt.gca().transAxes,
#              fontsize=12, verticalalignment='top', bbox=dict(facecolor='white', alpha=0.8))
    plt.legend()
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('ITO ')
    plt.show()